# **Setup**

In [175]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [184]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# **Import Data**

In [176]:

url='https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
df0=pd.read_csv(url)
df0.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [177]:
for c in df0.columns:
  print(df0[c].value_counts()) #get the basic information of all the columns

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64
-122.332653    265
-122.344896    254
-122.328079    252
-122.344997    239
-122.299160    231
              ... 
-122.322768      1
-122.288680      1
-122.405699      1
-122.323578      1
-122.343898      1
Name: X, Length: 23563, dtype: int64
47.708655    265
47.717173    254
47.604161    252
47.725036    239
47.579673    231
            ... 
47.556705      1
47.709101      1
47.513899      1
47.565438      1
47.563521      1
Name: Y, Length: 23839, dtype: int64
2047     1
1194     1
58550    1
64693    1
62644    1
        ..
96890    1
90745    1
92792    1
70263    1
2049     1
Name: OBJECTID, Length: 194673, dtype: int64
266238    1
81549     1
104088    1
126615    1
124566    1
         ..
164613    1
176899    1
178946    1
172801    1
295445    1
Name: INCKEY, Length: 194673, dtype: int64
266238    1
122129    1
111900    1
101659    1
99610     1
         ..
137750    1
131605    1
133652    1
143891    1
18441     1


# **Clean Data**

In [178]:
df0=df0[df0['STATUS']=='Matched'] #Only keep the data whose "STATUS" is "Matched" since "Unmatched" data are unreliable

In [179]:
df=df0[['SEVERITYCODE', 'ADDRTYPE', 'INATTENTIONIND', 'UNDERINFL', 'SDOT_COLCODE', 'SPEEDING']] 
df.head()

,SEVERITYCODE,ADDRTYPE,INATTENTIONIND,UNDERINFL,SDOT_COLCODE,SPEEDING
0,2,Intersection,NaN,N,11,NaN
1,1,Block,NaN,0,16,NaN
2,1,Block,NaN,0,14,NaN
3,1,Block,NaN,N,11,NaN
4,2,Intersection,NaN,0,11,NaN


In [180]:
for c in df.columns:
  print(df[c].value_counts()) 

1    132627
2     57159
Name: SEVERITYCODE, dtype: int64
Block           123663
Intersection     63559
Alley              747
Name: ADDRTYPE, dtype: int64
Y    29805
Name: INATTENTIONIND, dtype: int64
N    100274
0     80391
Y      5126
1      3995
Name: UNDERINFL, dtype: int64
11    83300
14    52757
16     9801
0      8978
28     8774
24     6390
13     5797
26     4657
18     3035
15     1575
12     1429
51     1293
29      473
21      181
56      178
27      162
54      134
23      123
48      106
31      103
25      101
34       92
64       75
69       68
33       53
55       50
66       23
22       17
32       12
53        9
44        8
61        7
35        6
58        5
68        4
36        4
46        3
52        2
47        1
Name: SDOT_COLCODE, dtype: int64
Y    9333
Name: SPEEDING, dtype: int64


In [181]:
#Fill empty cells
df['ADDRTYPE'].fillna('0', inplace=True)
df['INATTENTIONIND'].fillna('N', inplace=True)
df['SPEEDING'].fillna('N', inplace=True)
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,SEVERITYCODE,ADDRTYPE,INATTENTIONIND,UNDERINFL,SDOT_COLCODE,SPEEDING
0,2,Intersection,N,N,11,N
1,1,Block,N,0,16,N
2,1,Block,N,0,14,N
3,1,Block,N,N,11,N
4,2,Intersection,N,0,11,N


In [182]:
df.UNDERINFL.replace(to_replace=dict(N='0', Y='1'), inplace=True) # "0" should be interpreted as no, and "1" should be interpreted as yes. 
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,SEVERITYCODE,ADDRTYPE,INATTENTIONIND,UNDERINFL,SDOT_COLCODE,SPEEDING
0,2,Intersection,N,0,11,N
1,1,Block,N,0,16,N
2,1,Block,N,0,14,N
3,1,Block,N,0,11,N
4,2,Intersection,N,0,11,N


In [183]:
for c in df.columns:
  print(df[c].value_counts())

1    132627
2     57159
Name: SEVERITYCODE, dtype: int64
Block           123663
Intersection     63559
0                 1817
Alley              747
Name: ADDRTYPE, dtype: int64
N    159981
Y     29805
Name: INATTENTIONIND, dtype: int64
0    180665
1      9121
Name: UNDERINFL, dtype: int64
11    83300
14    52757
16     9801
0      8978
28     8774
24     6390
13     5797
26     4657
18     3035
15     1575
12     1429
51     1293
29      473
21      181
56      178
27      162
54      134
23      123
48      106
31      103
25      101
34       92
64       75
69       68
33       53
55       50
66       23
22       17
32       12
53        9
44        8
61        7
35        6
58        5
68        4
36        4
46        3
52        2
47        1
Name: SDOT_COLCODE, dtype: int64
N    180453
Y      9333
Name: SPEEDING, dtype: int64


# **Split the Data**

In [185]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

121462 train examples
30366 validation examples
37958 test examples


In [186]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('SEVERITYCODE')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

# **Setup Feature Columns**

In [187]:
feature_columns = []

In [188]:
indicator_column_names = ['ADDRTYPE', 'INATTENTIONIND', 'UNDERINFL', 'SDOT_COLCODE', 'SPEEDING']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, df[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [189]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)



# **Build the Model**

In [190]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
3774/3796 [============================>.] - ETA: 0s - loss: -866503.0000 - accuracy: 0.6979WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'ADDRTYPE': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=string>, 'INATTENTIONIND': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>, 'UNDERINFL': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=string>, 'SDOT_COLCODE': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=int64>, 'SPEEDING': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>}
Consider rewriting this model with the Functional API.
3796/3796 [==============================] - 8s 2ms/step - loss: -879780.3750 - accuracy: 0.6978 - val_loss: -3117290.5000 - val_accuracy: 0.6994
Epoch 2/100
3796/3796 [==============================] - 8s 2ms/step - loss: -9007711.0000 - a

# **Model Evaluation**

In [191]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1187/1187 [==============================] - 1s 1ms/step - loss: -702604181504.0000 - accuracy: 0.6989
Accuracy 0.6989040374755859
